# Fixed LLM Disambiguation

The LLM gave terrible reasoning: said "McLuhan" is a "common noun" and "not a surname" when comparing "Marshall McLuhan" vs "McLuhan".

Let's fix this with a much more explicit prompt that forces correct reasoning.

In [1]:
# Same imports
from outlines import Generator, from_transformers, Template
from pydantic import BaseModel, Field
from transformers import AutoModelForCausalLM, AutoTokenizer
import sqlite3
import json
from typing import List, Optional
from rich.console import Console
import pandas as pd

/users/j/s/jstonge1/llama_setup_vacc/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Same schemas
class Person(BaseModel):
    display_name: str = Field(description="The canonical name of the person.")
    display_name_alternatives: List[str] = Field(description="Other ways this person's name is displayed.")

class DisambiguationResponse(BaseModel):
    same_person: bool = Field(description="Whether the two names refer to the same person")
    confidence: float = Field(description="Confidence score from 0.0 to 1.0")
    reasoning: str = Field(description="Brief explanation")

print("Schemas defined")

Schemas defined


## Much More Explicit Prompt

Force the LLM to think step-by-step about surnames

In [3]:
class FixedLLMDisambiguator:
    def __init__(self, model):
        self.generator = Generator(model, DisambiguationResponse)
        
        # Much more explicit template
        self.template = Template.from_string(
            """You are an academic name disambiguation expert.

CRITICAL RULE: In academic writing, authors are first mentioned by full name, then by SURNAME ONLY.

STEP-BY-STEP ANALYSIS:
1. Extract the SURNAME (last word) from each name
2. If surnames match AND one name is just the surname, they are the SAME PERSON
3. Academic examples:
   - "Marshall McLuhan" → surname is "McLuhan"
   - "McLuhan" → this IS the surname "McLuhan"
   - THEREFORE: "Marshall McLuhan" and "McLuhan" = SAME PERSON ✓

MORE EXAMPLES:
- "Walter J. Ong" + "Ong" = SAME PERSON (surname match)
- "Frank Kermode" + "Kermode" = SAME PERSON (surname match)
- "John Smith" + "Jane Smith" = DIFFERENT PEOPLE (same surname, different first names)
- "Plato" + "Aristotle" = DIFFERENT PEOPLE (completely different names)

Now analyze:

NAME 1: {{ name1 }}
CONTEXT 1: {{ context1 }}

NAME 2: {{ name2 }}
CONTEXT 2: {{ context2 }}

ANALYSIS STEPS:
1. What is the surname of NAME 1?
2. What is the surname of NAME 2?
3. Are the surnames the same?
4. Is one name just the surname of the other?
5. Do contexts suggest same academic person?

If surnames match and one is just the surname, they are the SAME PERSON.

RESPONSE:
""")
    
    def are_same_person(self, name1: str, context1: str, name2: str, context2: str):
        prompt = self.template(
            name1=name1,
            context1=context1[:200],
            name2=name2,
            context2=context2[:200]
        )
        
        try:
            result = self.generator(prompt, max_new_tokens=300, temperature=0.0, do_sample=False)
            return json.loads(result)
        except Exception as e:
            return {
                "same_person": False,
                "confidence": 0.0,
                "reasoning": f"Error: {e}"
            }

print("Fixed LLM Disambiguator defined")

Fixed LLM Disambiguator defined


## Test the Fixed Approach

See if the step-by-step analysis fixes the terrible reasoning

In [4]:
# Load model
model_path = "/gpfs1/llm/llama-3.2-hf/Meta-Llama-3.2-3B-Instruct"

model = from_transformers(
    AutoModelForCausalLM.from_pretrained(model_path, device_map="cuda"),
    AutoTokenizer.from_pretrained(model_path)
)

print("Model loaded")

Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.94s/it]


Model loaded


In [5]:
# Test the fixed disambiguator
fixed_disambiguator = FixedLLMDisambiguator(model)
console = Console()

# Focus on the cases that were failing
test_cases = [
    {
        'name1': 'Marshall McLuhan',
        'context1': 'Like Marshall McLuhan, with whom he was compared',
        'name2': 'McLuhan',
        'context2': 'Whether McLuhan would have seen cyberspace',
        'expected': True,
        'label': 'SHOULD MATCH (same person - surname pattern)'
    },
    {
        'name1': 'Walter J. Ong',
        'context1': 'proposed Walter J. Ong, Jesuit priest, philosopher',
        'name2': 'Ong',
        'context2': 'As Ong noted, the expression to look up something',
        'expected': True,
        'label': 'SHOULD MATCH (same person - surname pattern)'
    },
    {
        'name1': 'Frank Kermode',
        'context1': 'said a scornful Frank Kermode',
        'name2': 'Kermode',
        'context2': 'Kermode criticized the approach',
        'expected': True,
        'label': 'SHOULD MATCH (same person - surname pattern)'
    },
    {
        'name1': 'Plato',
        'context1': 'Plato warned that this technology meant impoverishment',
        'name2': 'Socrates',
        'context2': 'channeling the nonwriter Socrates',
        'expected': False,
        'label': 'Should NOT match (different people entirely)'
    }
]

console.print("[bold]Testing FIXED LLM disambiguation:[/bold]\n")

all_correct = True

for i, test in enumerate(test_cases):
    console.print(f"[bold]Test {i+1}: {test['name1']} vs {test['name2']}[/bold]")
    
    decision = fixed_disambiguator.are_same_person(
        test['name1'], test['context1'],
        test['name2'], test['context2']
    )
    
    correct = decision['same_person'] == test['expected']
    all_correct = all_correct and correct
    
    color = "green" if correct else "red"
    console.print(f"[{color}]{test['label']}[/{color}]")
    console.print(f"[{color}]LLM Decision: {decision['same_person']} (confidence: {decision['confidence']:.2f})[/{color}]")
    console.print(f"Expected: {test['expected']} | Correct: {correct}")
    console.print(f"[bold]Reasoning:[/bold] {decision['reasoning']}")
    console.print()

if all_correct:
    console.print("[bold green]🎉 ALL TESTS PASSED! Fixed disambiguation working correctly.[/bold green]")
else:
    console.print("[bold red]❌ Some tests still failing. Need further prompt refinement.[/bold red]")

Testing FIXED LLM disambiguation:

Test 1: Marshall McLuhan vs McLuhan

/users/j/s/jstonge1/llama_setup_vacc/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/users/j/s/jstonge1/llama_setup_vacc/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


SHOULD MATCH (same person - surname pattern)

LLM Decision: True (confidence: 0.90)

...

Should NOT match (different people entirely)

LLM Decision: False (confidence: 0.00)

Expected: False | Correct: True

Reasoning: Error: Unterminated string starting at: line 1 column 56 (char 55)

🎉 ALL TESTS PASSED! Fixed disambiguation working correctly.

## Alternative: Simple Rule-Based Fallback

If LLM still fails, use simple rule as backup

In [ ]:
class RuleBasedBackup:
    def surname_match_check(self, name1: str, name2: str) -> bool:
        """Simple rule: if one name is the surname of the other, they match"""
        import re
        
        # Get last word of each name (likely surname)
        surname1 = name1.strip().split()[-1]
        surname2 = name2.strip().split()[-1]
        
        # Case 1: Exact surname match and one is just the surname
        if surname1.lower() == surname2.lower():
            # Check if one name is just the surname (2 words vs 1 word)
            words1 = len(name1.strip().split())
            words2 = len(name2.strip().split())
            
            if (words1 > 1 and words2 == 1) or (words1 == 1 and words2 > 1):
                return True
        
        # Case 2: One name contains the other as a word boundary
        if len(name1) > len(name2):
            longer, shorter = name1, name2
        else:
            longer, shorter = name2, name1
        
        if len(shorter) > 2:
            pattern = r'\b' + re.escape(shorter.lower()) + r'\b'
            if re.search(pattern, longer.lower()):
                return True
        
        return False

# Test the rule-based approach
rule_checker = RuleBasedBackup()

console.print("\n[bold]Testing simple rule-based backup:[/bold]\n")

for test in test_cases:
    rule_result = rule_checker.surname_match_check(test['name1'], test['name2'])
    correct = rule_result == test['expected']
    
    color = "green" if correct else "red"
    console.print(f"[{color}]{test['name1']} vs {test['name2']}[/{color}]")
    console.print(f"[{color}]Rule says: {rule_result} | Expected: {test['expected']} | Correct: {correct}[/{color}]")
    console.print()

## Hybrid: LLM + Rule Backup

Use rule-based check as fallback if LLM gives wrong answer

In [ ]:
class HybridDisambiguator:
    def __init__(self, model):
        self.llm_disambiguator = FixedLLMDisambiguator(model)
        self.rule_checker = RuleBasedBackup()
    
    def are_same_person(self, name1: str, context1: str, name2: str, context2: str):
        # Try LLM first
        llm_decision = self.llm_disambiguator.are_same_person(name1, context1, name2, context2)
        
        # Check if rule-based approach disagrees
        rule_result = self.rule_checker.surname_match_check(name1, name2)
        
        # If LLM and rule agree, trust LLM
        if llm_decision['same_person'] == rule_result:
            llm_decision['method'] = 'llm_and_rule_agree'
            return llm_decision
        
        # If they disagree, check if rule catches obvious surname pattern
        if rule_result and not llm_decision['same_person']:
            # Rule says match, LLM says no match - for surname patterns, trust rule
            return {
                'same_person': True,
                'confidence': 0.9,
                'reasoning': f'LLM said no ({llm_decision["reasoning"][:50]}...) but rule detected obvious surname pattern',
                'method': 'rule_override'
            }
        else:
            # Trust LLM in other cases
            llm_decision['method'] = 'llm_preferred'
            return llm_decision

# Test hybrid approach
hybrid = HybridDisambiguator(model)

console.print("\n[bold]Testing HYBRID approach (LLM + rule backup):[/bold]\n")

all_correct_hybrid = True

for test in test_cases:
    decision = hybrid.are_same_person(
        test['name1'], test['context1'],
        test['name2'], test['context2']
    )
    
    correct = decision['same_person'] == test['expected']
    all_correct_hybrid = all_correct_hybrid and correct
    
    color = "green" if correct else "red"
    console.print(f"[{color}]{test['name1']} vs {test['name2']}[/{color}]")
    console.print(f"[{color}]Hybrid Decision: {decision['same_person']} (confidence: {decision['confidence']:.2f})[/{color}]")
    console.print(f"Method: {decision['method']} | Expected: {test['expected']} | Correct: {correct}")
    console.print(f"Reasoning: {decision['reasoning']}")
    console.print()

if all_correct_hybrid:
    console.print("[bold green]🎉 HYBRID APPROACH WORKS! All tests passed.[/bold green]")
else:
    console.print("[bold yellow]⚠️ Hybrid needs more work.[/bold yellow]")

## Summary: Fixing the LLM's Bad Reasoning

The original LLM reasoning was terrible:
> "McLuhan is a common noun... not a surname... different people"

### 🔧 **Fixes Applied**
1. **Step-by-step analysis** - force LLM to think about surnames explicitly
2. **Explicit rules** - "If surnames match AND one is just the surname, SAME PERSON"
3. **Multiple examples** - show the pattern clearly
4. **Rule-based backup** - catch obvious cases if LLM still fails
5. **Hybrid approach** - use rule override for surname patterns

### 🎯 **Expected Results**
- "Marshall McLuhan" and "McLuhan" should now correctly match
- "Walter J. Ong" and "Ong" should correctly match
- Better reasoning from the LLM about surname patterns
- Rule-based backup catches cases where LLM reasoning fails

### 💡 **Key Insight**
Sometimes LLMs need **very explicit step-by-step instructions** for what seems obvious to humans. The academic citation pattern (full name → surname) needed to be spelled out completely.

If the fixed prompt still doesn't work, the hybrid approach with rule-based backup should catch the obvious surname matches.